In [1]:
import cv2
import ipywidgets as widgets
import threading
import time
from IPython.display import display

camera_widget = widgets.Image()#for camera widget

camera_index = 0 #according to your camera like if there are two camera both have different index
camera = cv2.VideoCapture(camera_index) #connection to specified camera

def update_camera_feed(): #for capturing and updating widget
    while True:
        ret, frame = camera.read() #ret is boolean value which tells weather image is captured or not and frame is the actual image
        if not ret:
            print("Error: Could not read frame.") #if image is not captured ret gives false indication
            break #bye bye update_camera_feed()
        
        _, jpeg = cv2.imencode('.jpg', frame) #it Convert the frame to JPEG format 
        camera_widget.value = jpeg.tobytes() #it updates the widget
        
        time.sleep(0.1)  # Add a delay to avoid overloading the kernel (your choice dpends on your system in mine it was working)


camera_thread = threading.Thread(target=update_camera_feed) # Start the camera feed thread it will keep calling this function
camera_thread.start()

display(camera_widget)# Display the camera feed in JupyterLab


Image(value=b'')

In [3]:
import torchvision.transforms as transforms
from src.dataset import ImageClassificationDataset

TASK = 'thumbs'  
CATEGORIES = ['thumbs_up', 'thumbs_down']
DATASETS = ['A', 'B']
#this is to transform the image before they are used by the model
TRANSFORMS = transforms.Compose([
    transforms.ColorJitter(0.2, 0.2, 0.2, 0.2),
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

datasets = {}
for name in DATASETS:
    datasets[name] = ImageClassificationDataset(TASK + '_' + name, CATEGORIES, TRANSFORMS) #ImageClassificationDataset categorizes all the data  

print(f"{TASK} task with {CATEGORIES} categories initialized.")


thumbs task with ['thumbs_up', 'thumbs_down'] categories initialized.


In [5]:
import ipywidgets as widgets
import cv2
import numpy as np
from IPython.display import display


dataset_widget = widgets.Dropdown(options=DATASETS, description='Dataset:')
category_widget = widgets.Dropdown(options=datasets[DATASETS[0]].categories, description='Category:')
count_widget = widgets.IntText(description='Count:')
save_widget = widgets.Button(description='Add')
refresh_widget = widgets.Button(description='Refresh Image')


def set_dataset(change):
    global dataset
    dataset = datasets[change['new']]
    count_widget.value = dataset.get_count(category_widget.value)


def update_counts(change):
    count_widget.value = dataset.get_count(change['new'])


def save(c):
    ret, frame = camera.read()
    if ret:
        datasets[dataset_widget.value].save_entry(frame, category_widget.value)
        count_widget.value = datasets[dataset_widget.value].get_count(category_widget.value)
    else:
        print("Error: No frame captured.")


def update_image(b):
    ret, frame = camera.read()
    if ret:
       
        _, jpeg = cv2.imencode('.jpg', frame)
        camera_widget.value = jpeg.tobytes()


dataset_widget.observe(set_dataset, names='value')
category_widget.observe(update_counts, names='value')
save_widget.on_click(save)
refresh_widget.on_click(update_image)


data_collection_widget = widgets.VBox([
    camera_widget,
    dataset_widget,
    category_widget,
    count_widget,
    save_widget,
    refresh_widget
])
print("data_collection_widget initiated")

data_collection_widget initiated


In [7]:
import torch
import torchvision
import ipywidgets as widgets

# Assuming datasets have already been defined and initialized.
# For example:
# datasets = { 'A': ImageClassificationDataset(...), 'B': ImageClassificationDataset(...) }
# Initialize dataset
dataset = datasets['A']  # Choose your dataset

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# ALEXNET
# model = torchvision.models.alexnet(pretrained=True)
# model.classifier[-1] = torch.nn.Linear(4096, len(dataset.categories))

# SQUEEZENET 
# model = torchvision.models.squeezenet1_1(pretrained=True)
# model.classifier[1] = torch.nn.Conv2d(512, len(dataset.categories), kernel_size=1)
# model.num_classes = len(dataset.categories)

# RESNET 18
model = torchvision.models.resnet18(pretrained=True)
model.fc = torch.nn.Linear(512, len(dataset.categories))

# RESNET 34
# model = torchvision.models.resnet34(pretrained=True)
# model.fc = torch.nn.Linear(512, len(dataset.categories))

model = model.to(device)

# Create UI widgets for saving/loading the model
model_save_button = widgets.Button(description='Save Model')
model_load_button = widgets.Button(description='Load Model')
model_path_widget = widgets.Text(description='Model Path', value='my_model.pth')

# Define functions for loading and saving the model
def load_model(c):
    model.load_state_dict(torch.load(model_path_widget.value))
    print("Model loaded from", model_path_widget.value)

model_load_button.on_click(load_model)

def save_model(c):
    torch.save(model.state_dict(), model_path_widget.value)
    print("Model saved to", model_path_widget.value)

model_save_button.on_click(save_model)

# Create a widget to display model operations
model_widget = widgets.VBox([
    model_path_widget,
    widgets.HBox([model_load_button, model_save_button])
])


print("Model configured and model_widget created")


C:\Users\LAVITRA\anaconda3\Lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
C:\Users\LAVITRA\anaconda3\Lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/resnet18-f37072fd.pth" to C:\Users\LAVITRA/.cache\torch\hub\checkpoints\resnet18-f37072fd.pth
100%|█████████████████████████████████████████████████████████████████████████████| 44.7M/44.7M [00:01<00:00, 24.0MB/s]


Model configured and model_widget created


In [9]:
import ipywidgets as widgets  # Ensure ipywidgets is imported
import time
import torch
import torch.nn.functional as F

# Assume model, dataset, and device are defined elsewhere in your code

BATCH_SIZE = 8
optimizer = torch.optim.Adam(model.parameters())
# optimizer = torch.optim.SGD(model.parameters(), lr=1e-3, momentum=0.9)

# Create widgets
epochs_widget = widgets.IntText(description='epochs', value=1)
eval_button = widgets.Button(description='evaluate')
train_button = widgets.Button(description='train')
loss_widget = widgets.FloatText(description='loss')
accuracy_widget = widgets.FloatText(description='accuracy')
progress_widget = widgets.FloatProgress(min=0.0, max=1.0, description='progress')

def train_eval(is_training):
    global BATCH_SIZE, model, dataset, optimizer, eval_button, train_button, accuracy_widget, loss_widget, progress_widget
    
    try:
        train_loader = torch.utils.data.DataLoader(
            dataset,
            batch_size=BATCH_SIZE,
            shuffle=True
        )

        train_button.disabled = True
        eval_button.disabled = True
        time.sleep(1)

        model.train() if is_training else model.eval()
        
        while epochs_widget.value > 0:
            i = 0
            sum_loss = 0.0
            error_count = 0.0
            
            for images, labels in iter(train_loader):
                # Send data to device
                images = images.to(device)
                labels = labels.to(device)

                if is_training:
                    optimizer.zero_grad()

                # Execute model to get outputs
                outputs = model(images)

                # Compute loss
                loss = F.cross_entropy(outputs, labels)

                if is_training:
                    loss.backward()
                    optimizer.step()

                # Increment progress
                error_count += len(torch.nonzero(outputs.argmax(1) - labels).flatten())
                count = len(labels.flatten())
                i += count
                sum_loss += float(loss)
                progress_widget.value = i / len(dataset)
                loss_widget.value = sum_loss / i
                accuracy_widget.value = 1.0 - error_count / i
                
            if is_training:
                epochs_widget.value -= 1
            else:
                break
    except Exception as e:
        print(f"An error occurred: {e}")

    model.eval()
    train_button.disabled = False
    eval_button.disabled = False

train_button.on_click(lambda c: train_eval(is_training=True))
eval_button.on_click(lambda c: train_eval(is_training=False))

train_eval_widget = widgets.VBox([
    epochs_widget,
    progress_widget,
    loss_widget,
    accuracy_widget,
    widgets.HBox([train_button, eval_button])
])

# display(train_eval_widget)
print("trainer configured and train_eval_widget created")


trainer configured and train_eval_widget created


In [11]:
import ipywidgets as widgets

# Define your previously created widgets here

data_live_box = widgets.HBox([data_collection_widget])
training_box = widgets.VBox([train_eval_widget, model_widget])

all_widget = widgets.VBox([data_live_box, training_box])

display(all_widget)
